# Derivation of the double pendulum
Consider the system, depicted below

![Sketch of the double pendulum system](system-sketch.svg)

The objective of this notebook is to derive the equations of motion of the system using [Lagrange's equations](https://en.wikipedia.org/wiki/Lagrangian_mechanics).  

In [1]:
using Pkg; Pkg.activate(".")
using SymPy
using LinearAlgebra

  Activating project at `C:\Users\fitzgeraldt\Documents\doublependulum\derivation`


In [2]:
dot(x,y) = sum( x .* y )
m1, m2, l1, l2, lp, g, Ig1, Ig2 = symbols("m_1, m_2, l_1, l_2, l_p, g, I_G1, I_G2", real=true, positive=true)
ϕ, t = symbols("phi, t", real=true)
θ1 = SymFunction("theta_1")(t)
θ2 = SymFunction("theta_2")(t)

ihat = [1, 0, 0]
jhat = [0, 1, 0]
khat = [0, 0, 1]
;

## Setup
The generalized coordinates are the absolute angles $q = [\theta_1,~ \theta_2]$.  The angle $\phi$ is fixed on the first body, and is the angle relative to the center of mass.

### The position vectors
All defined in the $\{\hat{i}\hat{j}\hat{k}\}$ system

In [3]:
r1 = l1*( sin(θ1)*ihat - cos(θ1)*jhat )
rp = lp*( sin(θ1+ϕ)*ihat - cos(θ1+ϕ)*jhat )
r2 = rp + l2*( sin(θ2)*ihat - cos(θ2)*jhat )
;

### The velocities

In [4]:
v1 = diff.( r1, t)

3-element Vector{Sym}:
 l_1*cos(theta_1(t))*Derivative(theta_1(t), t)
 l_1*sin(theta_1(t))*Derivative(theta_1(t), t)
                                             0

In [5]:
v2 = diff.( r2, t)

3-element Vector{Sym}:
 l_2*cos(theta_2(t))*Derivative(theta_2(t), t) + l_p*cos(phi + theta_1(t))*Derivative(theta_1(t), t)
 l_2*sin(theta_2(t))*Derivative(theta_2(t), t) + l_p*sin(phi + theta_1(t))*Derivative(theta_1(t), t)
                                                                                                   0

### The kinetic energy $T$

In [6]:
T1 = 1//2*m1*dot( v1, v1 ) + 1//2*Ig1*diff(θ1,t)^2 |> simplify

                                  2
/          2    \ /d             \ 
\I_G1 + l_1 *m_1/*|--(theta_1(t))| 
                  \dt            / 
-----------------------------------
                 2                 

In [7]:
T2 = 1//2*m2*dot( v2, v2 ) + 1//2*Ig2*diff(θ2,t)^2 |> simplify

                     2       /                     2                          
     /d             \        |   2 /d             \                           
I_G2*|--(theta_2(t))|    m_2*|l_2 *|--(theta_2(t))|  + 2*l_2*l_p*cos(phi + the
     \dt            /        \     \dt            /                           
---------------------- + -----------------------------------------------------
          2                                                                   

                                                                           2\
                      d              d                   2 /d             \ |
ta_1(t) - theta_2(t))*--(theta_1(t))*--(theta_2(t)) + l_p *|--(theta_1(t))| |
                      dt             dt                    \dt            / /
-----------------------------------------------------------------------------
           2                                                                 

In [8]:
T = T1 + T2 

                     2       /                     2                          
     /d             \        |   2 /d             \                           
I_G2*|--(theta_2(t))|    m_2*|l_2 *|--(theta_2(t))|  + 2*l_2*l_p*cos(phi + the
     \dt            /        \     \dt            /                           
---------------------- + -----------------------------------------------------
          2                                                                   

                                                                           2\ 
                      d              d                   2 /d             \ | 
ta_1(t) - theta_2(t))*--(theta_1(t))*--(theta_2(t)) + l_p *|--(theta_1(t))| | 
                      dt             dt                    \dt            / / 
----------------------------------------------------------------------------- 
           2                                                                  

                                    2
  /         

### The potential energy $V$

In [9]:
V1 = m1*g*dot(r1,jhat)

-g*l_1*m_1*cos(theta_1(t))

In [10]:
V2 = m2*g*dot(r2, jhat)

g*m_2*(-l_2*cos(theta_2(t)) - l_p*cos(phi + theta_1(t)))

In [11]:
V = V1 + V2 

-g*l_1*m_1*cos(theta_1(t)) + g*m_2*(-l_2*cos(theta_2(t)) - l_p*cos(phi + theta
_1(t)))

## Lagrange's equations

First, we build the Lagrangian $L = T-V$, and then use it in
$$ \frac{d}{dt}\left( \frac{\partial L}{\partial \dot{q}_k} \right) - \frac{\partial L}{\partial q_k} = 0, \quad k = 1,\,2$$



In [12]:
L = T-V |> simplify

                     2                                                        
     /d             \                                                         
I_G2*|--(theta_2(t))|                                                         
     \dt            /                                                         
---------------------- + g*l_1*m_1*cos(theta_1(t)) + g*m_2*(l_2*cos(theta_2(t)
          2                                                                   

                                     /                     2                  
                                     |   2 /d             \                   
                                 m_2*|l_2 *|--(theta_2(t))|  + 2*l_2*l_p*cos(p
                                     \     \dt            /                   
) + l_p*cos(phi + theta_1(t))) + ---------------------------------------------
                                                                              

                                                  

In [13]:
lag = [ diff(L, diff(q,t), t ) - diff( L, q ) |> simplify for q in [θ1, θ2] ]

2-element Vector{Sym}:
 I_G1*Derivative(theta_1(t), (t, 2)) + g*l_1*m_1*sin(theta_1(t)) + g*l_p*m_2*sin(phi + theta_1(t)) + l_1^2*m_1*Derivative(theta_1(t), (t, 2)) + l_2*l_p*m_2*sin(phi + theta_1(t) - theta_2(t))*Derivative(theta_2(t), t)^2 + l_2*l_p*m_2*cos(phi + theta_1(t) - theta_2(t))*Derivative(theta_2(t), (t, 2)) + l_p^2*m_2*Derivative(theta_1(t), (t, 2))
                                                                              I_G2*Derivative(theta_2(t), (t, 2)) + g*l_2*m_2*sin(theta_2(t)) + l_2^2*m_2*Derivative(theta_2(t), (t, 2)) - l_2*l_p*m_2*sin(phi + theta_1(t) - theta_2(t))*Derivative(theta_1(t), t)^2 + l_2*l_p*m_2*cos(phi + theta_1(t) - theta_2(t))*Derivative(theta_1(t), (t, 2))

## First order form
Let's work to rearrange this pair of coupled 2nd order equations to state-space (first order form) so we can use standard initial value ordinary differential equation solvers.
Let's pick the states
$$ z = [ \theta_1,\, \theta_2,\, \dot{\theta}_1,\, \dot{\theta}_2]^T $$


In [14]:
sol = solve( lag, [diff(θ1,t,t), diff(θ2, t, t)])

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall C:\Users\fitzgeraldt\.julia\packages\PyCall\L0fLP\src\numpy.jl:67


Dict{Any, Any} with 2 entries:
  Derivative(theta_2(t), (t, 2)) => -I_G1*g*l_2*m_2*sin(theta_2(t))/(I_G1*I_G2 …
  Derivative(theta_1(t), (t, 2)) => -I_G2*g*l_1*m_1*sin(theta_1(t))/(I_G1*I_G2 …

In [15]:
z1, z2, z3, z4 = symbols("z1, z2, z3, z4", real=true)
rule1 = Dict( diff(θ1,t) => z3, diff(θ2, t) => z4 )
rule2 = Dict( θ1 => z1, θ2 => z2 )

t1 = diff.( [θ1, θ2, diff(θ1,t), diff(θ2,t) ] )

4-element Vector{Sym}:
      Derivative(theta_1(t), t)
      Derivative(theta_2(t), t)
 Derivative(theta_1(t), (t, 2))
 Derivative(theta_2(t), (t, 2))

In [16]:
# Ugly and takes a long time to perform the simplify
#F = [ subs(eq,sol) |> subs(rule1) |> subs(rule2) |> simplify for eq in t1 ]

In order to avoid the nasty `simplify` in the above equations, let's pull out the mass matrix and then work with the resulting equations.  So let's reorganize the equations to look like

$$ M(q,\dot{q})\left[\begin{array}{c} \ddot{\theta}_1 \\ \ddot{\theta}_2 \end{array} \right] + h(q,\dot{q}) = 0 $$

In [17]:
M = [ diff( eq, diff(θ,t,t)) for eq in lag, θ in [θ1, θ2] ]

2×2 Matrix{Sym}:
                   I_G1 + l_1^2*m_1 + l_p^2*m_2  …  l_2*l_p*m_2*cos(phi + theta_1(t) - theta_2(t))
 l_2*l_p*m_2*cos(phi + theta_1(t) - theta_2(t))                                   I_G2 + l_2^2*m_2

In [18]:
h = simplify.( lag - M*diff.( [θ1,θ2], t, t) )

2-element Vector{Sym}:
 g*l_1*m_1*sin(theta_1(t)) + g*l_p*m_2*sin(phi + theta_1(t)) + l_2*l_p*m_2*sin(phi + theta_1(t) - theta_2(t))*Derivative(theta_2(t), t)^2
                                         l_2*m_2*(g*sin(theta_2(t)) - l_p*sin(phi + theta_1(t) - theta_2(t))*Derivative(theta_1(t), t)^2)

Now let's transform the coordinates to the states $z$

In [19]:
Mz = [ m |> subs(rule1) |> subs(rule2) for m in M]

2×2 Matrix{Sym}:
   I_G1 + l_1^2*m_1 + l_p^2*m_2  l_2*l_p*m_2*cos(phi + z1 - z2)
 l_2*l_p*m_2*cos(phi + z1 - z2)                I_G2 + l_2^2*m_2

In [20]:
hz = [ eq |> subs(rule1) |> subs(rule2) for eq in h ]

2-element Vector{Sym}:
 g*l_1*m_1*sin(z1) + g*l_p*m_2*sin(phi + z1) + l_2*l_p*m_2*z4^2*sin(phi + z1 - z2)
                                 l_2*m_2*(g*sin(z2) - l_p*z3^2*sin(phi + z1 - z2))

In [21]:
sympy.octave_code( Mz ) |> println

[I_G1 + l_1.^2.*m_1 + l_p.^2.*m_2 l_2.*l_p.*m_2.*cos(phi + z1 - z2); l_2.*l_p.*m_2.*cos(phi + z1 - z2) I_G2 + l_2.^2.*m_2]


In [22]:
sympy.octave_code( hz ) |> println

[g.*l_1.*m_1.*sin(z1) + g.*l_p.*m_2.*sin(phi + z1) + l_2.*l_p.*m_2.*z4.^2.*sin(phi + z1 - z2); l_2.*m_2.*(g.*sin(z2) - l_p.*z3.^2.*sin(phi + z1 - z2))]
